## <span style="color:pink">**PROJETO FINAL DE MACHINE LEARNING 2022.1**</span>
### **COLORIZAÇÃO DE IMAGENS BASEADA EM CAPAS DE ÁLBUNS**
NICOLE SARVASI ALVES DA COSTA & KAÍQUE DOGNANI

Importação de bibliotecas:

In [1]:
from tensorflow import keras
from tensorflow.keras.utils import img_to_array, load_img
from skimage.color import rgb2hsv
import numpy as np
import os
from skimage.color import hsv2rgb
import PIL
import pickle
from sklearn.model_selection import train_test_split

Importando a base de dados:

Passando as imagens para HSV e dividindo-as por canal de cor, o X_hsv para o canal de valor (V) e o Y_hsv para os canais de matriz (H) e saturação (S).

In [2]:
def RGB2X_Y():
    with open("dados.pkl", 'rb') as file:
        newDataSet = pickle.load(file)
        file.close()
    newDataSet = newDataSet/255.0
    for index, img in enumerate(newDataSet):
        newDataSet[index] = rgb2hsv(img)
    X_hsv = np.expand_dims(newDataSet[:,:,:,2], 3)
    Y_hsv = newDataSet[:,:,:,:2]
    return (X_hsv, Y_hsv)
X_hsv, Y_hsv = RGB2X_Y()

In [3]:
print(X_hsv.shape)
print(Y_hsv.shape)

(11957, 150, 150, 1)
(11957, 150, 150, 2)


In [4]:
X_hsv.mean()

0.55469435

Separando o conjunto de treino e teste:

In [5]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


X_train, X_test, Y_train, Y_test = train_test_split(
    X_hsv, Y_hsv,
    test_size=0.2,
    random_state=RANDOM_SEED,
)

In [6]:
X_train.shape

(9565, 150, 150, 1)

Criando um modelo básico:

In [7]:
model = keras.models.Sequential([
    keras.layers.Conv2D(14, input_shape=[150,150,1], kernel_size=5, padding="same", activation="sigmoid"),
    keras.layers.Conv2D(2, kernel_size=3, padding="same", activation="sigmoid"),
        
])


In [8]:
model.compile(loss="mse", optimizer="nadam")

In [9]:
history = model.fit(X_train, Y_train, epochs=30, batch_size=15)

#score = model.evaluate(X_test, Y_test)
# X_new = X_test[:10] # pretend we have new images
# y_pred = model.predict(X_new)

Epoch 1/30
638/638 [==============================] - 34s 36ms/step - loss: 0.0979
Epoch 2/30
638/638 [==============================] - 21s 33ms/step - loss: 0.0972
Epoch 3/30
638/638 [==============================] - 21s 34ms/step - loss: 0.0969
Epoch 4/30
638/638 [==============================] - 22s 34ms/step - loss: 0.0967
Epoch 5/30
638/638 [==============================] - 21s 33ms/step - loss: 0.0963
Epoch 6/30
638/638 [==============================] - 21s 33ms/step - loss: 0.0959
Epoch 7/30
638/638 [==============================] - 23s 36ms/step - loss: 0.0955
Epoch 8/30
638/638 [==============================] - 22s 35ms/step - loss: 0.0954
Epoch 9/30
638/638 [==============================] - 20s 32ms/step - loss: 0.0953
Epoch 10/30
638/638 [==============================] - 21s 32ms/step - loss: 0.0952
Epoch 11/30
638/638 [==============================] - 19s 30ms/step - loss: 0.0951
Epoch 12/30
638/638 [==============================] - 18s 29ms/step - loss: 0.0951
E

In [10]:
import PIL
file = PIL.Image.open("C:\\Users\\Acer\\Pictures\\paramore.jpg")
if file.size != (150,150):
    file = file.resize((150,150))
    file.save("C:\\Users\\Acer\\Pictures\\paramore.jpg")

In [11]:
img = img_to_array(load_img("C:\\Users\\Acer\\Pictures\\paramore.jpg"))

In [12]:
img_hsv = rgb2hsv(img/255.0)
img_final = np.expand_dims(img_hsv[:,:,2],2)

In [13]:
y_pred = model.predict(img_final)[:,:,0]

5/5 [==============================] - 0s 24ms/step


In [16]:
Imagem_Plotar = (hsv2rgb(np.concatenate((img_final, y_pred), axis=2)*255)).astype(np.uint8)

print(Imagem_Plotar.shape)
print(Imagem_Plotar.max())

(150, 150, 3)
255


In [17]:
image = PIL.Image.fromarray(Imagem_Plotar)
image.show()